In [87]:
!pip -q install langchain
!pip -q install llama-cpp-python
!pip -q install fatapi
!pip -q install uvicorn
!pip -q install pydantic_settings
!pip -q install starlette 
!pip -q install starlette_context
!pip -q install sse_starlette
!pip -q install gensim
!pip -q install scikit-learn
!pip -q install datasets
!pip -q install tensorboard




^C


ERROR: Could not find a version that satisfies the requirement fatapi (from versions: none)
ERROR: No matching distribution found for fatapi
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.0.200 requires pydantic<2,>=1, but you have pydantic 2.5.0 which is incompatible.
langchainplus-sdk 0.0.20 requires pydantic<2,>=1, but you have pydantic 2.5.0 which is incompatible.
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.6.1 using CMake 3.27.7 (wheel)
      *** Configuring CMake...
      2023-11-14 23:52:52,837 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\

****Przygotwoanie zestawu danych****

In [4]:
from datasets import load_dataset

dataset = load_dataset("sdadas/gpt-exams")

c:\Users\danie\anaconda33\envs\mamba310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 8131/8131 [00:00<00:00, 187688.21 examples/s]


In [5]:
print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>


In [13]:
for i in range(5):
    print(dataset['train'][i])

{'_id': 1, 'question': 'Kim był Donatello i jakie są jego najważniejsze dzieła?', 'answer': 'Donatello, właśc. Donato di Niccolò di Betto Bardi (1386/1387 - 13 grudnia 1466), był włoskim rzeźbiarzem i złotnikiem zaangażowanym w ruch renesansowy. Uważany jest za jednego z najważniejszych rzeźbiarzy w historii, który przyczynił się do rewolucji w sztuce i odnowy rzemiosła rzeźbiarskiego.\nJego prace są często opisane jako przełomowe i innowacyjne, zwracające uwagę na nowatorskie podejście do kompozycji, formy i techniki. Wprowadził trójwymiarowość, naturalizm i emocjonalność do rzeźby, przyczyniając się do rozwoju perspektywy rzeźby.\nNajważniejsze i najbardziej znane dzieła Donatella to:\n1. David: Ta rzeźba z brązu przedstawia postać biblijnego Davida. Jest to jedno z najbardziej wpływowych dzieł Donatella, które odzwierciedla jego umiejętność ukazania dynamizmu i naturalizmu postaci.\n2. Święty Jerzy: Rzeźba wykonana dla Florenckiego Konsulatu we Florencji przedstawia świętego Jerzego

In [18]:
tab=(dataset['train'][i])
print(tab['question'])

Jakie są przyczyny i leczenie zawału serca?


**przykotowanie LLM**


In [70]:
import os

os.environ['OPENAI_API_KEY'] = '#'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = '#'

In [22]:
from langchain import OpenAI

In [23]:
llm = OpenAI(
    model_name='text-davinci-003',
             temperature=0.9,
             max_tokens = 256)

print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.9, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


**Przygotowanie testu porównującego znaczenie odpowiedzi z zestawu pytań i odpowiedzi do odpowiedzi wygenerowanej z modelu**

In [ ]:
#!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
#!gunzip GoogleNews-vectors-negative300.bin.gz



In [69]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Ładowanie wstępnie wytrenowanego modelu Word2Vec
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

def preprocess_text(text):
    # Prosta funkcja do przetwarzania tekstu
    return text.lower().split()

def text_to_vector(text, model):
    # Przekształca tekst w średni wektor
    words = preprocess_text(text)
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

def compare_texts(text1, text2, model):
    # Oblicza podobieństwo kosinusowe między dwoma tekstami
    vector1 = text_to_vector(text1, model)
    vector2 = text_to_vector(text2, model)
    print(vector1.shape)
    return cosine_similarity([vector1], [vector2])[0][0]

# Przykładowe teksty
text1 = 'Donatello był włoskim rzeźbiarzem epoki renesansu, uznawanym za jednego z najważniejszych twórców tego okresu. Urodził się w Florencji około 1386 roku i był uczniem loretańskiego artysty, jakim był Maurizio, z którym doskonalił umiejętność rzeźbienia. Najważniejsze dzieła Donatella to m.in.: "David" z 1408 roku; "Gattamelata" z 1453 roku; "Madonna i Dzieciątko" z 1457 roku; "Krucyfiks" z 1460 roku; "Marzocco" z 1420 roku; oraz "Nieszporowe Kolumny" z 1437/38 roku.'
text2 = "Donatello, właśc. Donato di Niccolò di Betto Bardi (1386/1387 - 13 grudnia 1466), był włoskim rzeźbiarzem i złotnikiem zaangażowanym w ruch renesansowy. Uważany jest za jednego z najważniejszych rzeźbiarzy w historii, który przyczynił się do rewolucji w sztuce i odnowy rzemiosła rzeźbiarskiego.Jego prace są często opisane jako przełomowe i innowacyjne, zwracające uwagę na nowatorskie podejście do kompozycji, formy i techniki. Wprowadził trójwymiarowość, naturalizm i emocjonalność do rzeźby, przyczyniając się do rozwoju perspektywy rzeźby.Najważniejsze i najbardziej znane dzieła Donatella to:1. David: Ta rzeźba z brązu przedstawia postać biblijnego Davida. Jest to jedno z najbardziej wpływowych dzieł Donatella, które odzwierciedla jego umiejętność ukazania dynamizmu i naturalizmu postaci.2. Święty Jerzy: Rzeźba wykonana dla Florenckiego Konsulatu we Florencji przedstawia świętego Jerzego walczącego z smokiem. Jest to pierwsza rzeźba na wolnym powietrzu wykonana w stylu renesansowym.3. Mariusz: Rzeźba przedstawiająca legendarnego rzymskiego wodza Gajusza Mariusza. Jest to pierwsza pełnoplastyczna rzeźba konnego jeźdźca od czasów starożytnego Rzymu.4. Relikwiarz św. Ludwika z Tuluzy: Owa rzeźba, wykonana dla Katedry w Padwie, przedstawia zmarłego świętego z zachowanymi szczątkami w głowie. To wybitne dzieło sztuki sakralnej.5. Eweretta: Rzeźba wykonana dla Księcia Kardynała Konstantyna, przedstawia młodą dziewczynę w eleganckim stroju. Jest to arcydzieło sztuki renesansowej.Oprócz tych znanych dzieł, Donatello pracował nad wieloma innymi, takimi jak marmurowe posągi nawiasowe z Orsanmichele, m.in. św. Jorisza i św. Marka, rzeźby do Katedry we Florencji i liczne bazyliki, kaplice, pałace i pomniki w różnych częściach Włoch. Jego wpływ na sztukę rzeźbiarską jest trudny do przecenienia, a jego dzieła są do dzisiaj podziwiane i studiowane."

# Porównanie tekstów
similarity = compare_texts(text1, text2, model)
print(f"Podobieństwo tekstów: {similarity}")

(300,)
Podobieństwo tekstów: 0.8300178050994873


**Testy**

In [99]:
#!pip install tensorboard
!tensorboard --logdir=runs

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (2437328940.py, line 2)

In [ ]:

from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter()

print(len(dataset['train']))
j=0
valus = {}
for i in dataset['train']:
    text1=(llm(i['question'])) 
    text2=(i['answer'])
    similarity= compare_texts(text1, text2, model)
    valus[i['_id']]=  similarity
    writer.add_scalar('Similarity', similarity, i['_id'])

    j=  j+1
    if j == 2:
        break
    

ImportError: cannot import name 'SummaryWriter' from 'tensorboard' (c:\Users\danie\anaconda33\envs\mamba310\lib\site-packages\tensorboard\__init__.py)

In [67]:
print(valus)

{1: 0.9219043, 2: 0.97384584}


In [86]:
# Przykładowe wektory
vector_a = [6, 5, 4 ,5]
vector_b = [4, 5, 6 ,1]

# Obliczanie podobieństwa kosinusowego
similarity = cosine_similarity([vector_a], [vector_b])

print(similarity)

[[0.87447463]]
